## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=64
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict


In [6]:
### Read validation data
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

bkgnd_dict={}
num_bkgnd=500
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=[0,1,2,3]
### Extract validation data
for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
# del samples


/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy


In [7]:
df_runs=pd.DataFrame([])

In [8]:
### bkgnd: Get bkgnd chi-squares for different sigma wrt one standard sigma 
lst=[0.5,0.65,0.8,1.1]
param_label=0.65
for count,run in enumerate(lst):
    num=500
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_{0}_train_val.npy'.format(run)
    samples=np.load(fname,mmap_mode='r')[1000:1000+num][:,0,:,:]
    dict1={'label':'ref_%s'%(run),'fname':''}
    dict1['num_imgs']=num

    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(samples,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    
    df_runs=df_runs.append(dict1,ignore_index=True)


### Add pytorch runs

In [11]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
fldr='20210205_120608_cgan_predict_0.5_m2/'
parent_dir+=fldr
param_label=0.5
prefix='pred_%s_'%(param_label)
lst=[(9,23080),(12,29740), (15,36500), (14,34870)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    print(key)
    fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)




pred_0.5_9-23080
(1000, 128, 128)
0.9810246 -0.99782807
pred_0.5_12-29740
(1000, 128, 128)
0.98451245 -0.99669456
pred_0.5_15-36500
(1000, 128, 128)
0.99227196 -0.99521744
pred_0.5_14-34870
(1000, 128, 128)
0.9893413 -0.99713707


In [12]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
fldr='20210205_062936_cgan_predict_0.65_m2/'
parent_dir+=fldr
param_label=0.65
prefix='pred_%s_'%(param_label)
lst=[(13,32800),(11,27120),(13,31570), (14,33520), (14,34220)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    print(key)
    fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)




pred_0.65_13-32800
(1000, 128, 128)
0.9854815 -0.9973732
pred_0.65_11-27120
(1000, 128, 128)
0.9960004 -0.99870896
pred_0.65_13-31570
(1000, 128, 128)
0.98638237 -0.99762225
pred_0.65_14-33520
(1000, 128, 128)
0.986784 -0.9980204
pred_0.65_14-34220
(1000, 128, 128)
0.9844294 -0.99769014


In [13]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
fldr='20210205_122504_cgan_predict_0.8_m2/'
parent_dir+=fldr
param_label=0.8
prefix='pred_%s_'%(param_label)
lst=[(15,35620),(12,29490),(15,35860), (13,30950), (14,34380),(14,34840),(15,36320)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    print(key)
    fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


pred_0.8_15-35620
(1000, 128, 128)
0.99312824 -0.99868613
pred_0.8_12-29490
(1000, 128, 128)
0.9871935 -0.9990013
pred_0.8_15-35860
(1000, 128, 128)
0.9918073 -0.9974072
pred_0.8_13-30950
(1000, 128, 128)
0.9900308 -0.9992028
pred_0.8_14-34380
(1000, 128, 128)
0.9912386 -0.99823517
pred_0.8_14-34840
(1000, 128, 128)
0.9887223 -0.99895996
pred_0.8_15-36320
(1000, 128, 128)
0.9872924 -0.99888825


In [10]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
fldr='20210205_063709_cgan_predict_1.1_m2/'
parent_dir+=fldr
param_label=1.1
prefix='pred_%s_'%(param_label)
lst=[(9,23310),(11,27150), (11,28060), (13,31310), (13,32550), (14,35100), (15,35710)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    print(key)
    fname=glob.glob(parent_dir+'images/'+'inference_*label-{2}_epoch-{0}_step-{1}.npy'.format(epoch,step,param_label))[0]
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(bkgnd_dict[str(param_label)],dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



pred_1.1_9-23310
(1000, 128, 128)
0.9960735 -0.9997873
pred_1.1_11-27150
(1000, 128, 128)
0.9939979 -0.9994188
pred_1.1_11-28060
(1000, 128, 128)
0.99533325 -0.9996274
pred_1.1_13-31310
(1000, 128, 128)
0.99455374 -0.9993403
pred_1.1_13-32550
(1000, 128, 128)
0.9973461 -0.9989488
pred_1.1_14-35100
(1000, 128, 128)
0.995756 -0.99956375
pred_1.1_15-35710
(1000, 128, 128)
0.9961736 -0.99918497


In [14]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_imgs', 'spec_sdev', 'spec_val'],
      dtype='object')

In [15]:
col_list=['label', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','num_imgs','fname']

df_runs=df_runs.reindex(columns=col_list)


In [16]:
df_runs

,label,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,num_imgs,fname
0,ref_0.5,0.127021,0.083832,0.582520,30.617276,0.040159,0.003030,0.070477,0.805418,18.791174,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002444865431087542, 0.0016259722597047985, ...","[0.8476326735425547, 1.3200436841317889, 0.351...","[28033.485595391445, 14915.386892543762, 10076...","[63797.57892051363, 45566.12414815478, 33708.3...",500.0,
1,ref_0.65,0.001601,0.000148,0.003682,21.330048,0.000472,0.000982,0.000025,1.050824,0.132164,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0027666489046338618, 0.0018643363169184334,...","[0.9984811732297435, 1.1146344697956008, 0.298...","[40941.238924195844, 20243.500522244834, 12801...","[79758.82263291406, 55053.35863378426, 39483.1...",500.0,
2,ref_0.8,0.089498,0.043953,0.139352,28.184271,0.026129,0.019416,0.040195,1.165331,6.040435,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0029544550115642777, 0.0021164905080438326,...","[1.1204220686248407, 0.957873265788256, 0.2579...","[42986.27585849399, 23379.908991164513, 15311....","[87694.71658627165, 61310.9330808894, 45075.16...",500.0,
3,ref_1.1,0.486473,0.225602,0.784163,32.663335,0.130193,0.130679,0.210177,1.405127,42.548654,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003218997397904965, 0.002377549505076732, 0...","[1.2752818483821888, 0.7587410557036799, 0.211...","[57239.56950598676, 30804.390359674584, 20808....","[101921.86726250601, 73270.86392576854, 52459....",500.0,
4,pred_1.1_9-23310,0.417035,0.061495,1.672587,36.940652,0.198590,0.156950,0.008708,1.127243,51.256620,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003557584137026426, 0.002289949993937077, 0...","[1.2402649923782634, 0.7366977541116625, 0.242...","[123894.53289410842, 60498.63320809945, 37580....","[187677.68031174183, 118384.67919370288, 78277...",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pred_1.1_11-27150,0.298836,0.051568,1.141192,36.687946,0.184446,0.062821,0.008209,1.115004,31.772149,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003607840326361936, 0.002307881407439129, 0...","[1.2295968507057577, 0.7595981437939889, 0.240...","[124413.36316898563, 63356.50506298913, 36403....","[175841.03654097687, 114072.30754293311, 75402...",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pred_1.1_11-28060,0.195976,0.026880,1.192769,34.947190,0.125796,0.043300,0.005409,0.991590,56.391020,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0032230367759834093, 0.002163396401234748, ...","[1.2821427931478124, 0.7091872397856714, 0.219...","[105229.6822759246, 51327.79201993977, 30088.4...","[151372.91847709764, 97995.75950584018, 66318....",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pred_1.1_13-31310,0.102094,0.014086,0.624109,34.773038,0.055286,0.032721,0.002138,0.978852,25.223652,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0033639390620869495, 0.002247433157152555, ...","[1.2542559732364051, 0.755664489002774, 0.2262...","[105661.0828092424, 54135.38845974549, 31000.1...","[148600.8948388567, 94217.01908298703, 63046.7...",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pred_1.1_13-32550,0.058787,0.003467,0.450683,32.619873,0.032337,0.022983,0.001249,0.904210,23.336669,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0028497968538511867, 0.001930963211135437, ...","[1.283732360524966, 0.7333286545521616, 0.2125...","[90269.33782408084, 51019.334790182365, 30340....","[124649.87129491117, 87370.57075062857, 61150....",1000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,pred_1.1_14-35100,0.328338,0.111636,1.178292,37.057766,0.142812,0.073890,0.028785,1.212009,23.149109,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.004170480092597756, 0.0024487073238519982, ...","[1.1611711322217686, 0.838249902109113, 0.2605...","[136320.66336132822, 67938.52925774881, 37929....","

### Compare chi-squares

In [17]:
chi_keys=['label','chi_1','chi_spec1', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,ref_0.5,0.127021,0.582520,500.0,0.083832,0.040159,0.003030,0.070477,0.805418,18.791174,30.617276
1,ref_0.65,0.001601,0.003682,500.0,0.000148,0.000472,0.000982,0.000025,1.050824,0.132164,21.330048
2,ref_0.8,0.089498,0.139352,500.0,0.043953,0.026129,0.019416,0.040195,1.165331,6.040435,28.184271
3,ref_1.1,0.486473,0.784163,500.0,0.225602,0.130193,0.130679,0.210177,1.405127,42.548654,32.663335
4,pred_1.1_9-23310,0.417035,1.672587,1000.0,0.061495,0.198590,0.156950,0.008708,1.127243,51.256620,36.940652
5,pred_1.1_11-27150,0.298836,1.141192,1000.0,0.051568,0.184446,0.062821,0.008209,1.115004,31.772149,36.687946
6,pred_1.1_11-28060,0.195976,1.192769,1000.0,0.026880,0.125796,0.043300,0.005409,0.991590,56.391020,34.947190
7,pred_1.1_13-31310,0.102094,0.624109,1000.0,0.014086,0.055286,0.032721,0.002138,0.978852,25.223652,34.773038
8,pred_1.1_13-32550,0.058787,0.450683,1000.0,0.003467,0.032337,0.022983,0.001249,0.904210,23.336669,32.619873
9,pred_1.1_14-35100,0.328338,1.178292,1000.0,0.111636,0.142812,0.073890,0.028785,1.212009,23.149109,37.057766


## Plot

In [18]:
def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
#             yerr=np.sqrt((yerr2**2)/(y1**2) + (yerr1**2) * (yerr2**2/(yerr1**4)))/np.sqrt(row.num_imgs)
#             yerr=np.sqrt((np.abs(y))*((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs))
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

    
            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.legend()
            plt.title("relative spectrum")
        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        plt.xlim(0,img_size/2)


    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.title("relative spectrum")
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)    
    
    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.5, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})

    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [22]:
# Define the reference 
param_label=0.8
dict_val=bkgnd_dict[str(param_label)]

In [23]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('ref_0.5', 'ref_0.65', 'ref_0.8', 're…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [ ]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

### Test 

In [ ]:
df_runs.fname.values

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210112_191125_cgan_predict_1.1_m2/images/inference_spec_label-1.1_epoch-11_step-25999.npy'
a1=np.load(fname)
print(a1.shape)

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_1.1_train_val.npy'
val_arr=np.load(fname)[:2000][:,0,:,:]
print(val_arr.shape)

In [ ]:
f_pixel_intensity(f_invtransform(a1),normalize=True)
plt.xscale('symlog',linthreshx=50)


# img_lst=[a1]
# f_compare_pixel_intensity(img_lst,label_lst=['a1'],bkgnd_arr=val_arr)




In [ ]:
val_arr=val_arr[:,0,:,:]

In [ ]:
img_lst=[a1,val_arr]
f_compare_spectrum(img_lst=img_lst,label_lst=['a1'],bkgnd_arr=val_arr)


### Test histogram

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_5_4univ_cgan/norm_1_sig_0.5_train_val.npy'
print(fname)
samples=np.load(fname,mmap_mode='r')[-1000:][:,0,:,:]
print(samples.shape)

In [ ]:
# for loc in range(df_runs.shape[0]):
#     print(df_runs.loc[loc].hist_val[0:2],f_invtransform(df_runs.loc[loc].hist_bin_centers)[0:2])

In [ ]:
a1=f_invtransform(samples)

In [ ]:
np.max(samples),np.max(a1)

In [ ]:
### Read validation data
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
# bins=f_transform(bins)   ### scale to (-1,1)
# dict_val=f_compute_hist_spect(samples,bins)


In [ ]:
bins

In [ ]:
np.histogram(a1.flatten(),bins=bins,density=False)

In [ ]:
f_batch_histogram(a1,bins=bins,norm=True,hist_range=None)

In [ ]:
f_compute_hist_spect(a1,bins)['hist_val']

In [ ]:
f_compute_hist_spect(a1,bins)['hist_bin_centers']


In [ ]:
bins

In [ ]:
bkgnd_dict['0.5']['hist_val']
# f_invtransform(bkgnd_dict['0.5']['hist_bin_centers'])